In [1]:
from pathlib import Path

import fsspec
import s3fs
import xarray as xr
import zarr

In [8]:
file = 'https://hackathon-o.s3-ext.jc.rl.ac.uk/sim-data/DYAMOND3_example_data/healpix/toa_outgoing_longwave_flux/hpz10/glm.n2560_RAL3p3.apvera_20200120T00.toa_outgoing_longwave_flux.hpz10.zarr'
ds = xr.open_dataset(file,engine='zarr')

In [5]:
ds

<xarray.Dataset> Size: 1GB
Dimensions:                     (cell: 12582912, time: 12)
Coordinates:
  * cell                        (cell) int64 101MB 0 1 2 ... 12582910 12582911
    forecast_period             (time) timedelta64[ns] 96B ...
    forecast_reference_time     datetime64[ns] 8B ...
  * time                        (time) datetime64[ns] 96B 2020-01-20T00:02:00...
Data variables:
    toa_outgoing_longwave_flux  (time, cell) float64 1GB ...
Attributes:
    grid_mapping:     healpix_nested
    healpix_zoom:     10
    source:           Data from Met Office Unified Model
    standard_name:    toa_outgoing_longwave_flux
    um_stash_source:  m01s02i205
    um_version:       13.5
    units:            W m-2

In [7]:
s3cfg = dict([l.split(' = ') for l in Path('/home/markmuetz/.s3cfg').read_text().split('\n') if l])

In [8]:
jasmin_s3 = s3fs.S3FileSystem(
    anon=False, secret=s3cfg['secret_key'],
    key=s3cfg['access_key'],
    client_kwargs={'endpoint_url': 'https://hackathon-o.s3-ext.jc.rl.ac.uk'}
)
jasmin_s3.ls('data')

['data/20200101T0000Z_ph2496.pp',
 'data/DYAMOND3_example_data',
 'data/hpz0',
 'data/hpz10',
 'data/zarr_example.zarr']

In [8]:
store= s3fs.S3Map(root='s3://data/hpz0/glm.n2560_RAL3p3.apverd_20200120T00.air_temperature.hpz0.zarr', s3=jasmin_s3, check=False)


In [9]:
dsT2 = xr.open_zarr(store=store)

In [12]:
dsT2.air_temperature.sum().compute()

<xarray.DataArray 'air_temperature' ()> Size: 8B
array(376112.32569437)
Coordinates:
    forecast_reference_time  datetime64[ns] 8B 2020-01-20

In [14]:
# z10 store.
store= s3fs.S3Map(root='s3://data/hpz10/glm.n2560_RAL3p3.apverd_20200120T00.4.air_temperature.hpz10.zarr', s3=jasmin_s3, check=False)


In [15]:
dsTz10 = xr.open_zarr(store=store)

In [20]:
dsTz10.isel(time=0, pressure=-1).air_temperature.mean().compute()

<xarray.DataArray 'air_temperature' ()> Size: 8B
array(287.12667868)
Coordinates:
    forecast_period          timedelta64[ns] 8B NaT
    forecast_reference_time  datetime64[ns] 8B 2020-01-20
    pressure                 float32 4B 1e+03
    time                     datetime64[ns] 8B 2020-01-20

In [21]:
store= s3fs.S3Map(root='s3://data/DYAMOND3_example_data/healpix/toa_outgoing_longwave_flux/hpz10/glm.n2560_RAL3p3.apvera_20200120T00.toa_outgoing_longwave_flux.hpz10.nc', s3=jasmin_s3, check=False)

In [22]:
dsOLR2 = xr.open_zarr(store=store)

In [23]:
dsOLR2.load()

<xarray.Dataset> Size: 1GB
Dimensions:                     (cell: 12582912, time: 12)
Coordinates:
  * cell                        (cell) int64 101MB 0 1 2 ... 12582910 12582911
    forecast_period             (time) timedelta64[ns] 96B 00:01:59.999999970...
    forecast_reference_time     datetime64[ns] 8B 2020-01-20
  * time                        (time) datetime64[ns] 96B 2020-01-20T00:02:00...
Data variables:
    toa_outgoing_longwave_flux  (time, cell) float64 1GB 290.9 290.5 ... 201.1
Attributes:
    grid_mapping:     healpix_nested
    healpix_zoom:     10
    source:           Data from Met Office Unified Model
    standard_name:    toa_outgoing_longwave_flux
    um_stash_source:  m01s02i205
    um_version:       13.5
    units:            W m-2

In [25]:
dsOLR2.toa_outgoing_longwave_flux.mean()

<xarray.DataArray 'toa_outgoing_longwave_flux' ()> Size: 8B
array(235.82190961)
Coordinates:
    forecast_reference_time  datetime64[ns] 8B 2020-01-20

In [3]:
dsOLR3 = xr.open_dataset('s3://data/DYAMOND3_example_data/healpix/toa_outgoing_longwave_flux/hpz10/glm.n2560_RAL3p3.apvera_20200120T00.toa_outgoing_longwave_flux.hpz10.nc', engine='zarr')

NoCredentialsError: Unable to locate credentials

In [5]:
dsOLR3 = xr.open_dataset('s3://sim-data/DYAMOND3_example_data/healpix/toa_outgoing_longwave_flux/hpz10/glm.n2560_RAL3p3.apvera_20200120T00.toa_outgoing_longwave_flux.hpz10.zarr', engine='zarr')

NoCredentialsError: Unable to locate credentials

In [6]:
with jasmin_s3.open(
    's3://sim-data/DYAMOND3_example_data/healpix/toa_outgoing_longwave_flux/hpz10/glm.n2560_RAL3p3.apvera_20200120T00.toa_outgoing_longwave_flux.hpz10.nc/.zattrs',
    'rb'
) as f:
    print(f.read())

NameError: name 'jasmin_s3' is not defined

In [14]:
jasmin_s3 = s3fs.S3FileSystem(
    anon=True,
    client_kwargs={'endpoint_url': 'https://hackathon-o.s3-ext.jc.rl.ac.uk'}
)
print(jasmin_s3.ls('data'))

store= s3fs.S3Map(root='s3://data/DYAMOND3_example_data/healpix/toa_outgoing_longwave_flux/hpz10/glm.n2560_RAL3p3.apvera_20200120T00.toa_outgoing_longwave_flux.hpz10.nc', s3=jasmin_s3, check=False)

dsOLR2 = xr.open_zarr(store=store)

PermissionError: Access Denied

In [12]:
jasmin_s3 = s3fs.S3FileSystem(
    anon=True,
    client_kwargs={'endpoint_url': 'https://hackathon-o.s3-ext.jc.rl.ac.uk'}
)
jasmin_s3.ls('sim-data')

store= s3fs.S3Map(root='s3://sim-data/DYAMOND3_example_data/healpix/toa_outgoing_longwave_flux/hpz10/glm.n2560_RAL3p3.apvera_20200120T00.toa_outgoing_longwave_flux.hpz10.zarr', s3=jasmin_s3, check=False)

dsOLR2 = xr.open_zarr(store=store)

In [45]:
dsOLR2.isel(time=0).mean().compute()

<xarray.Dataset> Size: 32B
Dimensions:                     ()
Coordinates:
    forecast_period             timedelta64[ns] 8B 00:01:59.999999970
    forecast_reference_time     datetime64[ns] 8B 2020-01-20
    time                        datetime64[ns] 8B 2020-01-20T00:02:00
Data variables:
    toa_outgoing_longwave_flux  float64 8B 236.7

In [42]:
dsOLR2.nbytes / 1e9

1.308623048

In [18]:
import cartopy.crs as ccrs
import healpix as hp
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import easygems.healpix as egh
import easygems.remap as egr

ModuleNotFoundError: No module named 'easygems'

In [33]:
# Have to use access key/secret.
jasmin_s3 = s3fs.S3FileSystem(
    anon=True,
    client_kwargs={'endpoint_url': 'https://hackathon-o.s3-ext.jc.rl.ac.uk'}
)
jasmin_s3.ls('s3://sim-data/DYAMOND3_example_data/healpix/toa_outgoing_longwave_flux/hpz10/glm.n2560_RAL3p3.apvera_20200120T00.toa_outgoing_longwave_flux.hpz10.nc/.zattrs')

PermissionError: Access Denied